In [1]:
from pathlib import Path
import json
from datetime import datetime
from collections import namedtuple
import math
import pandas as pd  # pip install pandas
import numpy as np  # pip install numpy
import matplotlib.pyplot as plt  # pip install matplotlib
from scipy import stats
from scipy.stats import pearsonr  # pip install scipy

from tqdm.notebook import tqdm  # pip install tqdm
import plotly.express as px
import warnings
warnings.filterwarnings('ignore')


In [2]:
# ToDo list
# Create a function to compare regression results between multiple apps
# Create a function to select apps for analysis

df_lichess_reviews_raw = pd.read_json('DATAR/release_related/all_reviews/lichess-org-_-lichobile.json')
df_lichess_releases_raw = pd.read_json('DATAR/release_related/all_jsons/lichess-org-_-lichobile.json')
df_proton = pd.read_json('DATAR/release_related/all_reviews/ProtonVPN-_-android-app.json')

In [3]:
def preprocess_plotting(df: pd.DataFrame):
    """
    Takes a review dataframe and preprocesses it for plotting
    """
    df_scores_time = df[['score', 'at']].copy()

    # Adds a column to keep counts during grouping
    df_scores_time['count'] = 1

    # Add year and month to group on
    df_scores_time['year'] = pd.to_datetime(df_scores_time['at']).dt.year
    df_scores_time['month'] = pd.to_datetime(df_scores_time['at']).dt.month

    # Group by year -> month -> score
    df_scores_time = df_scores_time.groupby(by = ['year', 'month', 'score']).sum(numeric_only=True)
    df_grouped_scores_time = pd.DataFrame(df_scores_time['count'].index.to_list(), columns = ['year', 'month', 'score'])

    # Keep only the values
    df_grouped_scores_time['count'] = df_scores_time['count'].values
    
    # Add a datetime column to the values
    df_grouped_scores_time['day'] = 1
    df_grouped_scores_time['date'] = pd.to_datetime(df_grouped_scores_time[['day', 'month', 'year']])
    
    return df_grouped_scores_time

In [4]:
def calculate_rolling_average(df: pd.DataFrame):
    """
    Takes a review dataframe and preprocesses it for plotting
    ToDo: add some value for missing days, affects all apps but a big issue for apps with few reviews
    """
    df_scores_time = df[['score', 'at']].copy()
    
    # Adds a column to keep counts during grouping
    df_scores_time['count'] = 1

    # Add date to group on
    df_scores_time['year'] = pd.to_datetime(df_scores_time['at']).dt.year
    df_scores_time['month'] = pd.to_datetime(df_scores_time['at']).dt.month
    df_scores_time['day'] = pd.to_datetime(df_scores_time['at']).dt.day

    # Group on date
    df_scores_time['date'] = pd.to_datetime(df_scores_time[['year', 'month','day']])
    df_scores_time = df_scores_time.groupby(by = ['date']).sum()
    df_scores_time['mean'] = df_scores_time['score'] / df_scores_time['count']
    df_scores_time = df_scores_time.drop(columns = ['at', 'year', 'month', 'day', 'score', 'count'])

    # return rolling average
    df_rolling = df_scores_time.rolling(7).mean()
    df_rolling = df_rolling.iloc[6:]
    return df_rolling

In [5]:
def calculate_average(df: pd.DataFrame):
    """
    Takes a plotting dataframe and adds the average per month
    """
    # Initialize variables
    dict_average = {}
    Old_row_tuple = namedtuple('old_row', ['month', 'year'])
    old_row = Old_row_tuple(0, 0)
    total_score = 0
    total_reviews = 0

    for index, row in df.iterrows():
        # Is true if the new row is of the same month
        if (row.month, row.year) == (old_row.month, old_row.year):
            total_score += row.score * row['count']
            total_reviews += row['count']
            dict_average[(row.month, row.year)] = (total_score, total_reviews, total_score/total_reviews, row['date'])
            old_row = row
        # If the new row is the next month, create a new total score and total review
        else:
            total_score = row.score * row['count']
            total_reviews = row['count']
            dict_average[(row.month, row.year)] = (total_score, total_reviews, total_score/total_reviews, row['date'])
            old_row = row
            
    # Turns the dictionary into a dataframe        
    df_average = pd.DataFrame(dict_average).T
    df_average.columns = ['total score', 'total amount of reviews', 'average score', 'date']

    return df_average

In [6]:
def get_releases(df: pd.DataFrame):
    """ 
    Create a dataframe containing a release identifier and the release date
    Currently drops all releases made on the same day except for the last
    Releases are sorted from last to first    
    """
    # Select relevant columns
    df_releases = df[['google_play_tag', 'start_date']].copy()

    # Create a datetime dataframe to merge
    df_dates_temp = pd.DataFrame(columns=['year', 'month', 'day'])
    for n in range(len(df_releases)):    
        df_dates_temp.loc[n] = (list(map(int, df_releases['start_date'].values[n][:10].split('-'))))
    df_releases[['year', 'month', 'day']] = df_dates_temp

    # Add datetime, remove temporary columns
    df_releases['date'] = pd.to_datetime(df_releases[['year', 'month','day']])
    df_releases.index = df_releases['date']
    df_releases = df_releases.drop(columns = ['start_date', 'year', 'month', 'day', 'date'])
    df_releases = df_releases[df_releases.index.duplicated() == False]

    return df_releases

In [7]:
def combine_all_releases():
    """ 
    Combines all releases into a single dataframe and writes it to a csv
    Takes ~15 minutes
    """
    # Collect all Paths
    JSON_ROOT = Path('DATAR/release_related/all_jsons')
    json_paths = list(JSON_ROOT.glob('*.json'))

    # Create dataframe to store all releases of all apps
    datelist = pd.date_range(pd.to_datetime('2014-01-01'), periods=4038).tolist()
    df_releases_full = pd.DataFrame(index = datelist)

    # Populate datafrane with all apps, skip if there are no releases
    for json_path in tqdm(json_paths):
        app_releases = pd.read_json(json_path)
        if app_releases.shape == (0,0):
            continue
        else:
            df_releases_full[json_path.stem] = get_releases(app_releases)

    df_releases_full.to_csv('supplementary_data/releases_per_app')

# Needs the function if you don't have the csv yet
# combine_all_releases()
df_releases_full = pd.read_csv('supplementary_data/releases_per_app', index_col=0, dtype=str)

In [8]:
# Checks the amount of apps with only 1 release and removes them
one_release = 0
one_release_list = list()

for app in df_releases_full.columns:
    if df_releases_full[app].first_valid_index() == df_releases_full[app].last_valid_index():
        one_release_list.append(app)
        one_release += 1

df_releases_full = df_releases_full.drop(columns = one_release_list)
one_release

458

In [9]:
# Create a dataframe with the first and last valid date for each app to use later
# ToDo: will integrate this into find_treatment_control
df_begin_end_date = pd.DataFrame(columns = df_releases_full.columns)
df_begin_end_date.loc['first'] = df_releases_full.apply(pd.DataFrame.first_valid_index)
df_begin_end_date.loc['last'] = df_releases_full.apply(pd.DataFrame.last_valid_index)

In [10]:
def find_treatment_control(df_releases_full, df_begin_end_date, begin_date_temp, end_date_temp):
    """
    For a date range, compute the amount of valid apps with releases and without releases
    ToDo: flag apps with multiple releases and decide what to do with them
    """ 
    release_list = list()
    all_apps_list = list()

    for app in df_releases_full.columns:
        # Only consider valid apps which have had a release and not yet their last release in the full time frame
        if begin_date_temp > df_begin_end_date[app].loc['first'] and end_date_temp < df_begin_end_date[app].loc['last']:
            all_apps_list.append(app)
            # returns all releases for the app
            app_releases = df_releases_full[df_releases_full[app].notnull()][app].index.to_list()
            # Finds if the app has a release in the time frame
            if len([date for date in app_releases if begin_date_temp <= date <= end_date_temp]) > 0:
                release_list.append(app)

    # Find all apps which have no releases
    no_release_list = list(set(all_apps_list) - set(release_list))
    return no_release_list, release_list

In [ ]:
begin_date_temp = '2019-08-01'
end_date_temp = '2019-12-01'
no_release_list, release_list = find_treatment_control(df_releases_full, df_begin_end_date, begin_date_temp, end_date_temp)
(len(no_release_list), len(release_list))

(235, 96)

In [27]:
def find_groups_and_frames(df_releases_full, df_begin_end_date, frame_length):
    """
    Finds amount of apps in treatment and control groups for a given frame
    Takes ~12 minutes for a frame length of 10 -> scales linearly with frame length
    """
    begin_date_global = min(df_begin_end_date.loc['first'])
    end_date_global_adjusted = pd.to_datetime(max(df_begin_end_date.loc['last'])) - pd.DateOffset(days=frame_length)
    # list containing all dates
    date_list = [str(time)[:10] for time in pd.to_datetime(pd.date_range(start=begin_date_global, end=end_date_global_adjusted).tolist())]
    total_treat_control_list = list()
    progress_check = 0
    # Used to keep track of progress
    year = 2014

    for begin_date_temp in date_list:
        end_date_temp = str(pd.to_datetime(begin_date_temp) + pd.DateOffset(days=frame_length))[:10]
        no_release_list, release_list = find_treatment_control(df_releases_full, df_begin_end_date, begin_date_temp, end_date_temp)
        total_treat_control_list.append((len(no_release_list), len(release_list)))
        # Print update progress after a year
        progress_check += 1
        if progress_check % 365 == 0:
            year += 1
            print(year)

    # Write result to csv
    df_valid_releases = pd.DataFrame(total_treat_control_list, columns = ['control', 'treatment'], index = date_list)
    df_valid_releases['total'] = df_valid_releases['control'] + df_valid_releases['treatment']
    df_valid_releases.to_csv('supplementary_data/df_valid_releases_{}'.format(frame_length))

for group_length in [15, 30, 60, 90]:
    find_groups_and_frames(df_releases_full, df_begin_end_date, group_length)

2015
2016
2017
2018
2019
2020
2021
2022
2023
2015
2016
2017
2018
2019
2020
2021
2022
2023
2015
2016
2017
2018
2019
2020
2021
2022
2023
2015
2016
2017
2018
2019
2020
2021
2022
2023


In [39]:
df_valid_releases_10  = pd.read_csv('supplementary_data/df_valid_releases_10')
df_valid_releases_15  = pd.read_csv('supplementary_data/df_valid_releases_15')
df_valid_releases_30  = pd.read_csv('supplementary_data/df_valid_releases_30')
df_valid_releases_60  = pd.read_csv('supplementary_data/df_valid_releases_60')
df_valid_releases_90  = pd.read_csv('supplementary_data/df_valid_releases_90')

In [53]:
na_dict = dict()
for index in df_releases_full.index:
    na_dict[index] = sum(df_releases_full.loc[index].isnull())
min(na_dict.values())

848

In [40]:
df_valid_releases_10.describe()

,control,treatment,total
count,3602.000000,3602.000000,3602.000000
mean,219.735980,15.237646,234.973626
std,115.366265,10.684678,122.004061
min,0.000000,0.000000,0.000000
25%,112.000000,7.000000,123.000000
50%,258.000000,15.000000,277.000000
75%,328.000000,23.000000,349.000000
max,359.000000,48.000000,366.000000


In [29]:
df_valid_releases_15.describe()

,control,treatment,total
count,3597.000000,3597.000000,3597.000000
mean,213.199055,20.916597,234.115652
std,112.323377,14.370659,121.536007
min,0.000000,0.000000,0.000000
25%,110.000000,9.000000,122.000000
50%,249.000000,21.000000,276.000000
75%,318.000000,32.000000,347.000000
max,353.000000,59.000000,365.000000


In [30]:
df_valid_releases_30.describe()

,control,treatment,total
count,3582.000000,3582.000000,3582.000000
mean,196.865438,34.678113,231.543551
std,104.313820,23.211314,120.133295
min,0.000000,0.000000,0.000000
25%,105.000000,13.000000,120.000000
50%,229.500000,35.000000,274.000000
75%,293.000000,53.000000,344.000000
max,337.000000,88.000000,360.000000


In [31]:
df_valid_releases_60.describe()

,control,treatment,total
count,3552.000000,3552.000000,3552.000000
mean,172.767736,53.669482,226.437218
std,91.994811,34.716813,117.368128
min,0.000000,0.000000,0.000000
25%,90.000000,18.000000,117.000000
50%,198.000000,56.000000,269.000000
75%,258.000000,83.000000,337.000000
max,300.000000,121.000000,351.000000


In [36]:
df_valid_releases_90.describe()

,control,treatment,total
count,3522.000000,3522.000000,3522.000000
mean,154.880466,66.525270,221.405735
std,82.692902,42.072827,114.674258
min,0.000000,0.000000,0.000000
25%,83.000000,23.000000,114.250000
50%,171.000000,73.000000,263.500000
75%,228.750000,103.000000,330.000000
max,276.000000,144.000000,346.000000


In [35]:
df_valid_releases_90.loc[df_valid_releases_10['treatment'].idxmax()]
begin_date_temp = '2020-12-26'
end_date_temp = str(pd.to_datetime(begin_date_temp) + pd.DateOffset(days=90))[:10]
no_release_list, release_list = find_treatment_control(df_releases_full, df_begin_end_date, begin_date_temp, end_date_temp)
(len(no_release_list), len(release_list))

(181, 138)

In [13]:
def make_trend(df_rolling: pd.DataFrame, time_start: datetime, time_end: datetime):
    """ 
    Takes a dateframe containing a rolling (7 day) average per day, a start time and an end time.
    Returns a trend for the period between the start and end time.
    """
    # Select time period
    df_rolling = df_rolling[df_rolling.index > time_start]
    df_rolling = df_rolling[df_rolling.index < time_end]
    # Create trendline using linear regression
    slope, intercept, r, p, std_err = stats.linregress(list(range(len(df_rolling.values))), df_rolling['mean'].to_list())
    
    return slope, intercept

In [14]:
def compare_trends():
    return 0

In [15]:
def calculate_correlation():

    # Find timeframes

    # Split apps into treatment and control groups

    # Calculate trends

    # Calculate correlation
    correlation = 0
    return correlation

In [16]:
df_grouped = preprocess_plotting(df_lichess_reviews_raw)
df_rolling = calculate_rolling_average(df_lichess_reviews_raw)
df_average = calculate_average(df_grouped)
df_lichess_releases = get_releases(df_lichess_releases_raw)

Plottings historgrams for reviews

In [17]:
df_plot_1 = preprocess_plotting(df_lichess_reviews_raw)
df_plot_2 = preprocess_plotting(df_proton)

In [18]:
fig = px.histogram(df_plot_1, x = 'date', y = 'count', barmode = 'group', color = 'score')
fig.show()

In [19]:
fig = px.histogram(df_plot_2, x = 'date', y = 'count', barmode = 'group', color = 'score')
fig.show()

Plotting average score over time

In [20]:
fig = px.scatter(df_average, x= list(range(0,len(df_average))), y = 'average score')    # ToDo: change x to actual date
fig.show()

## Notes:
- Currently only the last release per day is kept
- There is a gap between 2014 and 2021 for releases
- Apps with one (458) or zero releases (42) are not considered
- I started at January 1 2014 for df_releases_full but there may be apps with releases before